# Apophis: 2020/2021 and 2029 predictions

In [ ]:
]st

In [ ]:
using Revise

In [ ]:
using JLD, PlanetaryEphemeris, Apophis, SPICE

In [ ]:
using TaylorIntegration

In [ ]:
#using LinearAlgebra: dot, cross, norm, diag
using LinearAlgebra
using Dates, Plots

In [ ]:
using Statistics

In [ ]:
dq = set_variables("ξ", order=5, numvars=7)

In [ ]:
fnamesseph = joinpath(Apophis.jldephpath, "sseph343ast_p21y_et.jld")
@time ss16asteph = load(fnamesseph, "ss16ast_eph");

In [ ]:
#apophis_eph_file = "Apophis_p24yo5v7_Quadmath.jld"
apophis_eph_file = "Apophis_p24y5o7v.jld"
tv1 = load(apophis_eph_file, "tv1")
xv1 = load(apophis_eph_file, "xv1");

In [ ]:
tvS1 = load(apophis_eph_file, "tvS1")
xvS1 = load(apophis_eph_file, "xvS1")
gvS1 = load(apophis_eph_file, "gvS1");

In [ ]:
x_Gamma_file = "x_Gamma.jld"
x_O7 = load(x_Gamma_file, "x_O7")
Γ_O7 = load(x_Gamma_file, "Γ_O7")
x_O6 = load(x_Gamma_file, "x_O6")
Γ_O6 = load(x_Gamma_file, "Γ_O6")
x_R = load(x_Gamma_file, "x_R")
Γ_R = load(x_Gamma_file, "Γ_R")
x_OR7 = load(x_Gamma_file, "x_OR7")
Γ_OR7 = load(x_Gamma_file, "Γ_OR7")
x_OR6 = load(x_Gamma_file, "x_OR6")
Γ_OR6 = load(x_Gamma_file, "Γ_OR6");

# Cholesky factorization of covariance matrix

In [ ]:
Hermitian(Γ_OR7) - Γ_OR7

In [ ]:
norm(Hermitian(Γ_OR7) - Γ_OR7)

In [ ]:
Γ_O6_cholf = cholesky(Hermitian(Γ_O6[1:6,1:6]))

In [ ]:
Γ_O7_cholf = cholesky(Hermitian(Γ_O7))

In [ ]:
Γ_R_cholf = cholesky(Hermitian(Γ_R))

In [ ]:
Γ_OR7_cholf = cholesky(Hermitian(Γ_OR7))

In [ ]:
Γ_OR6_cholf = cholesky(Hermitian(Γ_OR6[1:6,1:6]))

In [ ]:
Γ_O6_cU = Γ_O6_cholf.U
Γ_O6_cL = Γ_O6_cholf.L

Γ_O7_cU = Γ_O7_cholf.U
Γ_O7_cL = Γ_O7_cholf.L

Γ_R_cU = Γ_R_cholf.U
Γ_R_cL = Γ_R_cholf.L

Γ_OR7_cU = Γ_OR7_cholf.U
Γ_OR7_cL = Γ_OR7_cholf.L

Γ_OR6_cU = Γ_OR6_cholf.U
Γ_OR6_cL = Γ_OR6_cholf.L;

In [ ]:
x_OR7

In [ ]:
σ_OR7 = sqrt.(diag(Γ_OR7))

In [ ]:
sqrt.(diag(Γ_OR7)) .* vcat(1e-8ones(6), 1e-14)

## $\Gamma$ matrix eigenvalues/vectors

In [ ]:
# Compute nominal initial condition in au, au/day, au/day^2
q0_OR7 = Apophis.initialcond(vcat(1e-8ones(6), 1e-14).*x_OR7)

In [ ]:
# relative scalings analogous to Milani (2005)
scaling_milani_2005 = vcat(  (1/norm(q0_OR7[1:3]))*ones(3), (1/norm(q0_OR7[4:6]))*ones(3), 1/norm(q0_OR7[7])  )

In [ ]:
# jet transport propagation scaling
jt_scaling = vcat(1e-8ones(6), 1e-14)

In [ ]:
# compute cov matrix with relative scalings
Γ_OR7_scm05 = diagm(scaling_milani_2005) * ( diagm(jt_scaling)*Γ_OR7*diagm(jt_scaling) ) * diagm(scaling_milani_2005)

In [ ]:
# compute normal matrix with relative scalings
C_OR7_scm05 = inv(Γ_OR7_scm05)

In [ ]:
# compute eigenvalues, eigenvectors of normal matrix with relative scalings
C_eig_OR7_scm05 = eigen(C_OR7_scm05)

In [ ]:
# revert eigenvector of "weak" direction to JT scaling
C_eig_OR7_scm05.vectors[:,1]./scaling_milani_2005./jt_scaling

### Generating samples from cov matrices

Confidence ellipsoids are defined by
$$
(\tilde{x}-x^\star)^T C (\tilde{x}-x^\star) = \sigma^2
$$
where $C$ is the normal matrix (essentially, the Hessian) with size $N\times N$, where $N$ is the number of degrees of freedom of the orbit determination. That is,  $N=6$ when dealing only with positions and velocities, and $N=7$ when dealing with positions, velocities and $A_2$, i.e., the Yarkovsky non-gravitational parameter. The covariance matrix is $\Gamma=C^{-1}$.


According to Bierman (1977) do this using the square-root information matrix., in order to sample an ensemble of initial conditions with zero mean and covariance $\Gamma$, we may perform a Cholesky factorization on $\Gamma$:
$$
\Gamma = L R
$$
where $L$ and $R$ are called, respectively, the left and right factors of the Cholesky factorization. The $L$ and $R$ matrices are, respectively, lower and upper triangular matrices. Now, let $w$ denote a zero-mean, unit standard-deviation random $N$-vector (i.e., an $N$-vector whose components are each a random number sampled from a normal distribution). Further, let $\hat{x}$ denote the nominal solution. Then
$$
\hat{x} + L \cdot w
$$
is a random $N$-vector with mean equal to the nominal solution and covariance $\Gamma$ (Bierman, 1977; Chodas and Yeomans, 1999).  Thus, in order to generate random samples from the covariance matrix, it suffices to multiply the left Cholesky factor by 6-vectors of normal random samples, as done below.

$\Gamma_{new} = J^T \Gamma * J$

### 6-variable, optical-only covariance matrix

In [ ]:
Hermitian(Γ_O6)

In [ ]:
# recovering the covariance matrix from cholesky(Γ).L*randn(6)
cov(map(x->Γ_O6_cholf.L*randn(6), 1:100000))

### 7-variable, optical+radar covariance matrix

In [ ]:
Hermitian(Γ_OR7)

In [ ]:
# recovering the covariance matrix from cholesky(Γ).L*randn(7)
cov(map(x->Γ_OR7_cholf.L*randn(7), 1:100000))

# Construction of B-plane (2029 close approach)

In [ ]:
t_2029 = tvS1[47];

In [ ]:
t_2029()

In [ ]:
jd0 = datetime2julian(DateTime(2008,9,24))

In [ ]:
ss16asteph.t0

In [ ]:
const Nm1 = size(ss16asteph.x)[2] ÷ 6
const N = Nm1 + 1

In [ ]:
ss16asteph_2029 = ss16asteph((jd0-J2000+t_2029)*daysec)
xe_2029 = ss16asteph_2029[union(3ea-2:3ea,3(N-1+ea)-2:3(N-1+ea))];

In [ ]:
apophis_2029 = xvS1[47,1:6];

In [ ]:
xae_2029 = apophis_2029-xe_2029;

In [ ]:
# Apophis geocentric range at 2029 closest approach (au)
rae_2029 = sqrt(xae_2029[1]^2 + xae_2029[2]^2 + xae_2029[3]^2); #(A2_JPL)

In [ ]:
a_2029 = semimajoraxis(xae_2029...,μ[ea], 0.0);

In [ ]:
# asymptotic inbound velocity v_\infty (au/day)
v_infty = sqrt(μ[ea]/(-a_2029));

In [ ]:
# h = r × v
hvec_2029 = cross(xae_2029[1:3], xae_2029[4:6]);

In [ ]:
h_2029 = sqrt(hvec_2029[1]^2 + hvec_2029[2]^2 + hvec_2029[3]^2);

In [ ]:
# Laplace-Runge-Lenz (eccentricity) vector
# \vec{e} = (\vec{v} × \vec{h})/μ - \vec{r}/r
evec_2029 = cross(xae_2029[4:6], hvec_2029)/μ[ea] - xae_2029[1:3]/rae_2029;

In [ ]:
# osculating eccentricity
e_2029 = eccentricity(xae_2029..., μ[ea], 0.0);

In [ ]:
# osculating eccentricity
i_2029 = inclination(xae_2029...);

In [ ]:
# periapsis position (unit vector)
P_v = evec_2029./e_2029;

In [ ]:
# periapsis velocity (unit vector)
Q_v = cross(hvec_2029, P_v)./h_2029;

In [ ]:
P_v__ = xae_2029[1:3]/rae_2029;
Q_v__ = xae_2029[4:6]/sqrt(xae_2029[4]^2 + xae_2029[5]^2 + xae_2029[6]^2);

In [ ]:
P_v(x_OR7)

In [ ]:
P_v__(x_OR7)

In [ ]:
Q_v(x_OR7)

In [ ]:
Q_v__(x_OR7)

In [ ]:
rad2deg(acos(dot(P_v, Q_v)(x_OR7)))  # should be 90 deg, or pretty close

In [ ]:
rad2deg(acos(dot(P_v__, Q_v__)(x_OR7))) # should be 90 deg, or pretty close

In [ ]:
#inbound asymptote direction 
S_v = (P_v + (sqrt(e_2029^2 - 1))Q_v)/e_2029;
#S_v()

In [ ]:
# B-vector: "vector from the planet center to the intersection between the B-plane and the asymptote"
Bvec_2029 = cross(S_v, hvec_2029)./v_infty;

In [ ]:
# Earth impact cross section; "critical" B
# derived from conservation of energy and angular momentum
#  = what impact parameter B corresponds to a grazing impact, given planet radius and mass, and asymptotic vel v_infty?
# if actual B is equal or less to this, then impact happens
b_E_2029 = sqrt( 1 + (2μ[ea])/((RE/au)*(v_infty)^2) );

In [ ]:
# Sun's barycentric cartesian state at Apophis 2029 CA as a function of A2
xs_2029 = ss16asteph_2029[union(3su-2:3su,3(N-1+su)-2:3(N-1+su))];

In [ ]:
# Earth's heliocentric cartesian state at Apophis 2029 CA as a function of A2
xes_2029 = xe_2029 - xs_2029;

In [ ]:
# Earth's osculating semimajor axis at nominal 2029 CA
a_E_2029 = semimajoraxis(xes_2029..., μ[su], 0.0)(x_OR7)

In [ ]:
# Earth's heliocentric velocity at Apophis 2029 CA as a function of A2 (au/day)
v_pl_2029 = xes_2029[4:6];

In [ ]:
#ξ-axis is essentially the MOID (Valsecchi et al, 2003)
ξ_v_unnormalized = cross(v_pl_2029, S_v)
ξ_v_norm = sqrt(ξ_v_unnormalized[1]^2 + ξ_v_unnormalized[2]^2 + ξ_v_unnormalized[3]^2)
ξ_v = ξ_v_unnormalized./ξ_v_norm;

In [ ]:
#ζ-axis: delay/advance in CA time (Valsecchi et al, 2003)
ζ_v = -cross(S_v, ξ_v);

In [ ]:
B_dot_ξ = dot(Bvec_2029, ξ_v);

In [ ]:
B_dot_ζ = dot(Bvec_2029, ζ_v);

In [ ]:
B_dot_ξ(x_OR7)*au/RE

In [ ]:
B_dot_ζ(x_OR7)*au/RE

In [ ]:
norm(Bvec_2029(x_OR7))*au/RE

# Valsecchi circles

We compute Valsecchi circles as a function of the components of the unperturbed planetocentric velocity vector $\mathbf{U}$ of Apophis in a coordinate frame $(X,Y,Z)$ such that the $Y$-axis is along the heliocentric velocity vector of the planet. It is enough to have the $U_y=U\cos\theta$ component and the norm $U=|\mathbf{U}|$, where $\theta$ is the angle between the $Y$-axis and $\mathbf{U}$. Furthermore, $\mathbf{U}$ should be in units such that the planet's heliocentric velocity is 1.

In [ ]:
x_sol_Val_circ = x_OR7

In [ ]:
S_v_nom = S_v(x_sol_Val_circ) # small-body unperturbed planetocentric velocity unit vector

In [ ]:
norm(S_v_nom)

In [ ]:
res_2029 = xes_2029(x_sol_Val_circ)[1:3] # Earth's heliocentric radius, au
res_2029_norm = norm(res_2029) # Earth's heliocentric range, au
res_2029_unit = res_2029/res_2029_norm # Earth's heliocentric radius unit vector
res_2029_norm

In [ ]:
ves_2029 = xes_2029(x_OR7)[4:6]*au/daysec # Earth's velocity, km/s
ves_2029_norm = norm(ves_2029) # Earth's speed, km/s

In [ ]:
ves_2029_unit = ves_2029/ves_2029_norm # Earth's velocity unit vector

In [ ]:
# X-Y angle (degrees)
rad2deg(acos(dot(ves_2029_unit, res_2029_unit)))

In [ ]:
# angle between Y-axis and \vec{U}
cosθ_ = dot(S_v_nom, ves_2029_unit)

In [ ]:
v_infty_kms = v_infty(x_OR7)*au/daysec # Apophis unperturbed speed, km/sec

In [ ]:
# The norm of \vec{U} in appropriate units
U_norm_ = v_infty_kms/ves_2029_norm

In [ ]:
# U_y
U_y_ = U_norm_*cosθ_

In [ ]:
R__, D__ = valsecchi_circle(U_y_, U_norm_, 7, 6)
D_Valsecchi_7_6 = D__/(PlanetaryEphemeris.RE/(res_2029_norm*au))
R_Valsecchi_7_6 = R__/(PlanetaryEphemeris.RE/(res_2029_norm*au))
D_Valsecchi_7_6, R_Valsecchi_7_6, D_Valsecchi_7_6-R_Valsecchi_7_6

In [ ]:
R_, D_ = valsecchi_circle(U_y_, U_norm_, 6, 5)
D_Valsecchi_6_5 = D_/(PlanetaryEphemeris.RE/(res_2029_norm*au))
R_Valsecchi_6_5 = R_/(PlanetaryEphemeris.RE/(res_2029_norm*au))
D_Valsecchi_6_5, R_Valsecchi_6_5, D_Valsecchi_6_5-R_Valsecchi_6_5

In [ ]:
R_, D_ = valsecchi_circle(U_y_, U_norm_, 8, 7)
D_Valsecchi_8_7 = D_/(PlanetaryEphemeris.RE/(res_2029_norm*au))
R_Valsecchi_8_7 = R_/(PlanetaryEphemeris.RE/(res_2029_norm*au))
D_Valsecchi_8_7, R_Valsecchi_8_7, D_Valsecchi_8_7-R_Valsecchi_8_7

In [ ]:
A2_JPL = -5.592840054057059
σ_A2_JPL = 2.201

In [ ]:
npoints = 3000
# note: no sigma factor needed here, since enclosing volume already contains ~3-sigma
rndvv = [randn(7) for i in 1:npoints];

In [ ]:
# sample uniformly random points on N-hypersphere
# code borrowed from http://corysimon.github.io/articles/uniformdistn-on-sphere/
function rand_sph(ndimsphere)
    v = zeros(ndimsphere)  # initialize so we go into the while loop
    while norm(v) < 0.0001
        v = randn(ndimsphere)
    end
    return v/norm(v)  # normalize to unit norm and return
end

In [ ]:
# generate `npoints` points on 7-dim hypersphere
#rndvv = [sigma_factor*rand_sph(7) for i in 1:npoints];

In [ ]:
dxv_O6 = map(x->(x_O6 .+ vcat(Γ_O6_cL*x[1:6],0.0) ), rndvv)
dxv_O7 = map(x->(x_O7 .+ Γ_O7_cL*x ), rndvv)
dxv_R = map(x->(x_R .+ Γ_R_cL*x ), rndvv)
dxv_OR7 = map(x->(x_OR7 .+ Γ_OR7_cL*x ), rndvv)
dxv_OR6 = map(x->(x_OR6 .+ vcat(Γ_OR6_cL*x[1:6],0.0) ), rndvv);

In [ ]:
# optical-only solution, no Yarkovsky
B_dot_ξv_O6 = B_dot_ξ.(dxv_O6)
B_dot_ζv_O6 = B_dot_ζ.(dxv_O6)

# optical-only solution
B_dot_ξv_O7 = B_dot_ξ.(dxv_O7)
B_dot_ζv_O7 = B_dot_ζ.(dxv_O7)

# radar-only solution
B_dot_ξv_R = B_dot_ξ.(dxv_R)
B_dot_ζv_R = B_dot_ζ.(dxv_R)

# optical+radar solution
B_dot_ξv_OR7 = B_dot_ξ.(dxv_OR7)
B_dot_ζv_OR7 = B_dot_ζ.(dxv_OR7)

# optical+radar solution, no Yarkovsky
B_dot_ξv_OR6 = B_dot_ξ.(dxv_OR6)
B_dot_ζv_OR6 = B_dot_ζ.(dxv_OR6)

# optical+radar nominal solution
B_dot_ξ0 = B_dot_ξ(x_OR7)
B_dot_ζ0 = B_dot_ζ(x_OR7);

In [ ]:
BB_OR7 = sqrt.(B_dot_ξv_OR7.^2 + B_dot_ζv_OR7.^2)*au/RE;

In [ ]:
# impact probability a la Monte Carlo
count(x->x < b_E_2029(), BB_OR7)/length(BB_OR7)

In [ ]:
#?count

In [ ]:
# Jacobian of transformation from initial conditions to B.ξ, B.ζ coordinates on target plane
t_x2B = hcat(TaylorSeries.gradient(B_dot_ξ), TaylorSeries.gradient(B_dot_ζ));
size(t_x2B)

In [ ]:
# transform cov matrix to B-plane coords
Γ_B_O6 = Hermitian(t_x2B(x_O6)[1:6,:]'*Γ_O6[1:6,1:6]*(t_x2B(x_O6)[1:6,:]))
# get cholesky-L factor
L_B_O6 = cholesky(Γ_B_O6).L 

# repeat for each solution and corresponding cov matrix

Γ_B_O7 = Hermitian(t_x2B(x_O7)'*Γ_O7*(t_x2B(x_O7)))
L_B_O7 = cholesky(Γ_B_O7).L

Γ_B_R = Hermitian(t_x2B(x_R)'*Γ_R*(t_x2B(x_R)))
L_B_R = cholesky(Γ_B_R).L

Γ_B_OR7 = Hermitian(t_x2B(x_OR7)'*Γ_OR7*(t_x2B(x_OR7)))
L_B_OR7 = cholesky(Γ_B_OR7).L

Γ_B_OR6 = Hermitian(t_x2B(x_OR6)[1:6,:]'*Γ_OR6[1:6,1:6]*(t_x2B(x_OR6)[1:6,:]))
L_B_OR6 = cholesky(Γ_B_OR6).L;

In [ ]:
sigma_factor = 3
sigma_factor_5 = 5

In [ ]:
angv = vcat(0:0.0125:2pi, 0);

In [ ]:
# sample points on 2-dim unit sphere

#rndvv_2sph = [sigma_factor*rand_sph(2) for i in 1:500]
#rndv_2sph_ξ = [rndvv_2sph[i][1] for i in 1:size(rndvv_2sph)[1]]
#rndv_2sph_ζ = [rndvv_2sph[i][2] for i in 1:size(rndvv_2sph)[1]];

# on 2-dim unit sphere (unit 1-sphere?), things are as easy as sampling the (3) unit circle...
rndv_2sph_ξ = sigma_factor*cos.(angv)
rndv_2sph_ζ = sigma_factor*sin.(angv);

rndv_2sph_ξ_5 = sigma_factor_5*cos.(angv)
rndv_2sph_ζ_5 = sigma_factor_5*sin.(angv);

In [ ]:
# then, project points from circle with radius 3 (i.e., locus of 3-sigma random points) onto B-plane coordinates

B_ξ_3σ_ell_O6 = (B_dot_ξ(x_O6) .+ L_B_O6[1,1]*rndv_2sph_ξ .+ L_B_O6[1,2]*rndv_2sph_ζ)*au/RE
B_ζ_3σ_ell_O6 = (B_dot_ζ(x_O6) .+ L_B_O6[2,1]*rndv_2sph_ξ .+ L_B_O6[2,2]*rndv_2sph_ζ)*au/RE

B_ξ_3σ_ell_O7 = (B_dot_ξ(x_O7) .+ L_B_O7[1,1]*rndv_2sph_ξ .+ L_B_O7[1,2]*rndv_2sph_ζ)*au/RE
B_ζ_3σ_ell_O7 = (B_dot_ζ(x_O7) .+ L_B_O7[2,1]*rndv_2sph_ξ .+ L_B_O7[2,2]*rndv_2sph_ζ)*au/RE

B_ξ_3σ_ell_R = (B_dot_ξ(x_R) .+ L_B_R[1,1]*rndv_2sph_ξ .+ L_B_R[1,2]*rndv_2sph_ζ)*au/RE
B_ζ_3σ_ell_R = (B_dot_ζ(x_R) .+ L_B_R[2,1]*rndv_2sph_ξ .+ L_B_R[2,2]*rndv_2sph_ζ)*au/RE

B_ξ_3σ_ell_OR7 = (B_dot_ξ(x_OR7) .+ L_B_OR7[1,1]*rndv_2sph_ξ .+ L_B_OR7[1,2]*rndv_2sph_ζ)*au/RE
B_ζ_3σ_ell_OR7 = (B_dot_ζ(x_OR7) .+ L_B_OR7[2,1]*rndv_2sph_ξ .+ L_B_OR7[2,2]*rndv_2sph_ζ)*au/RE

B_ξ_3σ_ell_OR6 = (B_dot_ξ(x_OR6) .+ L_B_OR6[1,1]*rndv_2sph_ξ .+ L_B_OR6[1,2]*rndv_2sph_ζ)*au/RE
B_ζ_3σ_ell_OR6 = (B_dot_ζ(x_OR6) .+ L_B_OR6[2,1]*rndv_2sph_ξ .+ L_B_OR6[2,2]*rndv_2sph_ζ)*au/RE

# 5-sigma
B_ξ_5σ_ell_OR7 = (B_dot_ξ(x_OR7) .+ L_B_OR7[1,1]*rndv_2sph_ξ_5 .+ L_B_OR7[1,2]*rndv_2sph_ζ_5)*au/RE
B_ζ_5σ_ell_OR7 = (B_dot_ζ(x_OR7) .+ L_B_OR7[2,1]*rndv_2sph_ξ_5 .+ L_B_OR7[2,2]*rndv_2sph_ζ_5)*au/RE;

In [ ]:
plotlyjs()
#gr()
#pgfplotsx()

In [ ]:
scatter(
    B_dot_ξv_OR7*au/RE,
    B_dot_ζv_OR7*au/RE,
    label="JTMC, OR7",
    marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_OR7,
    B_ζ_3σ_ell_OR7,
    label="3-σ, OR7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_5σ_ell_OR7,
    B_ζ_5σ_ell_OR7,
    label="5-σ, OR7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!([B_dot_ξ0*au/RE],
    [B_dot_ζ0*au/RE],
    label="nominal, OR7",
    marker=:circle, ms=3.0,
)
scatter!(
    B_dot_ξv_OR6*au/RE,
    B_dot_ζv_OR6*au/RE,
    label="JTMC, OR6 (A2=0)",
    marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_OR6,
    B_ζ_3σ_ell_OR6,
    label="3-σ, OR6 (A2=0)",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!([B_dot_ξ(x_OR6)*au/RE],
    [B_dot_ζ(x_OR6)*au/RE],
    label="nominal, OR6 (A2=0)",
    marker=:circle, ms=3.0,
)
plot!(cos.(angv), sin.(angv), label="Earth radius")
plot!(b_E_2029()*cos.(angv), b_E_2029()*sin.(angv), label="Earth cross section")
plot!(
    R_Valsecchi_8_7*cos.(angv), 
    D_Valsecchi_8_7.+R_Valsecchi_8_7*sin.(angv), 
    label="8:7 res. Valsecchi circle",
    #aspectratio=0.8,#1,
    #legend=false,
)
plot!(
    R_Valsecchi_7_6*cos.(angv), 
    D_Valsecchi_7_6.+R_Valsecchi_7_6*sin.(angv), 
    label="7:6 res. Valsecchi circle",
)
plot!(
    R_Valsecchi_6_5*cos.(angv), 
    D_Valsecchi_6_5.+R_Valsecchi_6_5*sin.(angv), 
    label="6:5 res. Valsecchi circle",
    aspectratio=0.8,
    #leg=false,
    legend=:outertopright,
    #legend=:topright,
    #legendfontsize=8
)
xlabel!("B.ξ [Earth radii]")
ylabel!("B.ζ [Earth radii]")
title!("B-plane (Öpik's frame)")
xlims!(-5, 5)
xticks!(-5:5)
ylims!(-3, 9)
yticks!(-3:9)

In [ ]:
plotlyjs()

In [ ]:
scatter(
    B_dot_ξv_O7*au/RE, 
    B_dot_ζv_O7*au/RE, 
    label="JTMC, O7", 
    marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!(
    B_dot_ξv_R*au/RE, 
    B_dot_ζv_R*au/RE, 
    label="JTMC, R7", 
    marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!(
    B_dot_ξv_OR7*au/RE,
    B_dot_ζv_OR7*au/RE,
    label="JTMC, OR7",
    marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_O7,
    B_ζ_3σ_ell_O7,
    label="3-σ, O7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_R,
    B_ζ_3σ_ell_R,
    label="3-σ, R7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_OR7,
    B_ζ_3σ_ell_OR7,
    label="3-σ, OR7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!([B_dot_ξ0*au/RE],
    [B_dot_ζ0*au/RE],
    label="nominal, OR7",
    marker=:circle, ms=3.0,
)
#plot!(cos.(angv), sin.(angv), label="Earth radius")
#plot!(b_E_2029()*cos.(angv), b_E_2029()*sin.(angv), label="Earth cross section")
plot!(
    R_Valsecchi_8_7*cos.(angv), 
    D_Valsecchi_8_7.+R_Valsecchi_8_7*sin.(angv), 
    label="8:7 res. Valsecchi circle",
    #legend=false,
)
plot!(
    R_Valsecchi_7_6*cos.(angv), 
    D_Valsecchi_7_6.+R_Valsecchi_7_6*sin.(angv), 
    label="7:6 res. Valsecchi circle",
)
plot!(
    R_Valsecchi_6_5*cos.(angv), 
    D_Valsecchi_6_5.+R_Valsecchi_6_5*sin.(angv), 
    label="6:5 res. Valsecchi circle",
    #aspectratio=0.8, #1
    #leg=false,
    legend=:outertopright,
    #legend=:topright,
    #legendfontsize=8
)
xlabel!("B.ξ [Earth radii]")
ylabel!("B.ζ [Earth radii]")
title!("B-plane (Öpik's frame)")
xlims!(1.47,1.505)
ylims!(4,12)

- compare $\chi^2$ a la Milani vs jet transport `inv(TaylorSeries.hessian(Q))` CHECK
- integrate with `Quadmath.Float128` CHECK (things are essentially the same)
- characterize 2029 impact initial conditions --> after fixing sampling from covariance matrix and after including optical obs, uncertainty region shrinks: no impact initial conditions remaining, even for optical-only orbit

In [ ]:
x_OR7[7] # A_2 O+R fit

In [ ]:
sqrt(Γ_OR7[7,7]) # σ_{A_2} O+R fit

In [ ]:
gr()

In [ ]:
# "Equivalent of Fig 1 in Farnocchia et al. (2013) Icarus, 224 192-200)"
# T = Tholen et al. (2013)
# R = radar (Giorgini 2008 + Brozovic 2018)
# T ellipsoid is expected to be roughly equivalent to Farnocchia's T ellipsoid,
# because dynamical models, astrometry and optical set and weights are the same; also A2 is assumed =0.
# T+R ellipsoid is not expected to be equivalent to Farnocchia's, since we include all radar astrometry through 2005-2013
# 7:6 and 8:7 Valsecchi circles, which enclose the ellipsoids, are out of the scale, so not shown

scatter(
    B_dot_ξv_O6*au, 
    B_dot_ζv_O6*au, 
    label="JTMC, O6 (A2=0)", 
    marker=:circle, ms=1.0, msc=:auto, # match marker stroke color
    markerstrokewidth = 0
)
scatter!(
    B_dot_ξv_OR6*au, 
    B_dot_ζv_OR6*au,
    label="JTMC, OR6 (A2=0)", 
    marker=:circle, ms=1.0, msc=:auto, # match marker stroke color
    markerstrokewidth = 0
)
scatter!(
    B_dot_ξv_OR7*au, 
    B_dot_ζv_OR7*au, 
    label="JTMC, OR7",  # A2=-0.82±2.8 10^{-14} au/day^2 (1-sigma)
    marker=:circle, ms=1.0, msc=:auto, # match marker stroke color
    markerstrokewidth = 0
)
plot!(
    B_ξ_3σ_ell_O6*RE,
    B_ζ_3σ_ell_O6*RE,
    label="3-σ, O6 (A2=0)",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_OR6*RE,
    B_ζ_3σ_ell_OR6*RE,
    label="3-σ, OR6 (A2=0)",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
plot!(
    B_ξ_3σ_ell_OR7*RE,
    B_ζ_3σ_ell_OR7*RE,
    label="3-σ, OR7",
    #marker=:circle, ms=1.0, markerstrokewidth = 0, msc=:auto # match marker stroke color
)
scatter!(
    [B_dot_ξ(x_O6)*au],
    [B_dot_ζ(x_O6)*au],
    label="nominal, O6 (A2=0)",
    marker=:circle,
)
scatter!(
    [B_dot_ξ(x_OR6)*au],
    [B_dot_ζ(x_OR6)*au],
    label="nominal, OR6 (A2=0)",
    marker=:circle,
)
scatter!(
    [B_dot_ξ(x_OR7)*au],
    [B_dot_ζ(x_OR7)*au],
    label="nominal, OR7",
    marker=:circle,
    #legend=false,
    legend=:outertopright,
    #legend=:bottomright
)
#plot!(
#    RE*R_Valsecchi_7_6*cos.(angv),
#    RE*D_Valsecchi_7_6.+RE*R_Valsecchi_7_6*sin.(angv),
#    label="7:6 res. Valsecchi circle",
#)
#plot!(
#    RE*R_Valsecchi_8_7*cos.(angv),
#    RE*D_Valsecchi_8_7.+RE*R_Valsecchi_8_7*sin.(angv),
#    label="8:7 res. Valsecchi circle",
#)
xlabel!("B.ξ [km]")
ylabel!("B.ζ [km]")
#xlims!(9465,9520)
#ylims!(4.67e4, 4.86e4)
#yticks!(4.68e4:0.02e4:4.86e4)
#xticks!(9465:5:9520)

In [ ]:
x0_JPL = vcat(zeros(6), A2_JPL)

In [ ]:
julian2datetime(jd0+t_2029(x0_JPL))

In [ ]:
au*rae_2029(x0_JPL)

In [ ]:
julian2datetime.(jd0 .+ tvS1(vcat(zeros(6), 0.0)))[end-30:end]

In [ ]:
t_2020 = tvS1[27]
t_2021 = tvS1[29]
t_2029 = tvS1[47];

In [ ]:
julian2datetime(jd0+t_2020(x0_JPL))

In [ ]:
julian2datetime(jd0+t_2021(x0_JPL))

In [ ]:
julian2datetime(jd0+t_2029(x0_JPL))

In [ ]:
ss16asteph_2020 = ss16asteph((jd0-J2000+t_2020)*daysec)
xe_2020 = ss16asteph_2020[union(3ea-2:3ea,3(N-1+ea)-2:3(N-1+ea))];

ss16asteph_2021 = ss16asteph((jd0-J2000+t_2021)*daysec)
xe_2021 = ss16asteph_2021[union(3ea-2:3ea,3(N-1+ea)-2:3(N-1+ea))];

ss16asteph_2029 = ss16asteph((jd0-J2000+t_2029)*daysec)
xe_2029 = ss16asteph_2029[union(3ea-2:3ea,3(N-1+ea)-2:3(N-1+ea))];

In [ ]:
apophis_2020 = xvS1[27,1:6]
apophis_2021 = xvS1[29,1:6]
apophis_2029 = xvS1[47,1:6]
xae_2020 = apophis_2020-xe_2020
xae_2021 = apophis_2021-xe_2021
xae_2029 = apophis_2029-xe_2029;

In [ ]:
rae_2020 = sqrt(xae_2020[1]^2 + xae_2020[2]^2 + xae_2020[3]^2);

In [ ]:
rae_2021 = sqrt(xae_2021[1]^2 + xae_2021[2]^2 + xae_2021[3]^2);

In [ ]:
rae_2029 = sqrt(xae_2029[1]^2 + xae_2029[2]^2 + xae_2029[3]^2);

In [ ]:
au*(rae_2020(x_OR7) - 0.216276663586499)

In [ ]:
au*(rae_2021(x_OR7) - 0.11265113489653)

In [ ]:
au*(rae_2029(x_OR7) - 0.000252172816142565)

In [ ]:
julian2datetime(2462240.407032288)

In [ ]:
#
0.21593875724774*60/3

In [ ]:
0.000252172816142565*au

In [ ]:
d_CA_2029_JPL = 0.000252172816142565
d_CA_2029_JPL_min = 0.000247515097583323
d_CA_2029_JPL_max = 0.000256837212164523

In [ ]:
(d_CA_2029_JPL-d_CA_2029_JPL_min)*au/3, (d_CA_2029_JPL-d_CA_2029_JPL_max)*au/3

In [ ]:
julian2datetime(jd0+t_2029(x_OR6))

In [ ]:
julian2datetime(jd0+t_2029(x_OR7))

In [ ]:
au*rae_2029(x_OR7)

In [ ]:
au*rae_2029(x_OR6)

In [ ]:
au*(rae_2020(x0_JPL) - 0.216276663586499)

In [ ]:
au*(rae_2021(x0_JPL) - 0.11265113489653)

In [ ]:
au*(rae_2029(x0_JPL) - 0.000252172816142565)

In [ ]:
#t_2020

In [ ]:
#rae_2020

In [ ]:
#t_CA = t_2020
#d_CA = rae_2020

t_CA = t_2029
d_CA = rae_2029;

In [ ]:
# Jacobian of transformation from initial conditions to d_CA_2020, t_CA_2020
t_x2dtCA = hcat(TaylorSeries.gradient(d_CA), TaylorSeries.gradient(t_CA));
size(t_x2dtCA)

In [ ]:
# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_dtCA_OR7 = Hermitian(t_x2dtCA(x_OR7)'*Γ_OR7*(t_x2dtCA(x_OR7)))
Γ_dtCA_OR6 = Hermitian(t_x2dtCA(x_OR6)[1:6,:]'*Γ_OR6*(t_x2dtCA(x_OR6)[1:6,:]))
# get cholesky-L factor
L_dtCA_OR7 = cholesky(Γ_dtCA_OR7).L
L_dtCA_OR6 = cholesky(Γ_dtCA_OR6).L

In [ ]:
sqrt.(diag(Γ_dtCA_OR6)).*[au,daysec]

In [ ]:
sqrt.(diag(Γ_dtCA_OR7)).*[au,daysec]

In [ ]:
dt_v_OR6 = Matrix{Float64}(undef, 1000, 2)
dt_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    dt_v_OR6[i,:] = L_dtCA_OR6*randn(2)
    dt_v_OR7[i,:] = L_dtCA_OR7*randn(2)
end

In [ ]:
d_3σ_ell_OR6 = (L_dtCA_OR6[1,1]*rndv_2sph_ξ .+ L_dtCA_OR6[1,2]*rndv_2sph_ζ) # .+ d_CA(x_OR6)
t_3σ_ell_OR6 = (L_dtCA_OR6[2,1]*rndv_2sph_ξ .+ L_dtCA_OR6[2,2]*rndv_2sph_ζ) # .+ t_CA(x_OR6)

d_3σ_ell_OR7 = (L_dtCA_OR7[1,1]*rndv_2sph_ξ .+ L_dtCA_OR7[1,2]*rndv_2sph_ζ) # .+ d_CA(x_OR7)
t_3σ_ell_OR7 = (L_dtCA_OR7[2,1]*rndv_2sph_ξ .+ L_dtCA_OR7[2,2]*rndv_2sph_ζ) # .+ t_CA(x_OR7)

Δd_OR76 = d_CA(x_OR7)-d_CA(x_OR6)
Δt_OR76 = t_CA(x_OR7)-t_CA(x_OR6)
;

In [ ]:
#plotly()

In [ ]:
scatter(
    (Δd_OR76.+dt_v_OR7[:,1])*au, 
    (Δt_OR76.+dt_v_OR7[:,2])*daysec, 
    ms=0.1, msc=:auto, label="OR7 (JT sampling)",
    legend=:outertopright
)
scatter!(dt_v_OR6[:,1]*au, dt_v_OR6[:,2]*daysec, ms=0.1, msc=:auto, label="OR6 (JT sampling)")
plot!(d_3σ_ell_OR6*au, t_3σ_ell_OR6*daysec, label="OR6 (3-σ)")
plot!((Δd_OR76.+d_3σ_ell_OR7)*au, (Δt_OR76.+t_3σ_ell_OR7)*daysec, label="OR7 (3-σ)")
scatter!([Δd_OR76*au], [Δt_OR76*daysec], label="OR7, nominal")
xlabel!("Geocentric range at closest approach [km]")
ylabel!("Time of closest approach [s]")
yca = year(julian2datetime(jd0 + t_CA(x_OR7)))
mca = monthname(julian2datetime(jd0 + t_CA(x_OR7)))
title!("$mca $yca close approach: CA time and distance")

In [ ]:
d_CA(x_OR6), julian2datetime(jd0 + t_CA(x_OR6))

In [ ]:
au*d_CA(x_OR6)

In [ ]:
d_CA(x_OR7), julian2datetime(jd0 + t_CA(x_OR7))

In [ ]:
au*0.000252172816142565

In [ ]:
au*d_CA(x_OR7)

# 2020 RA/DEC predictions

In [ ]:
file_dict = load("predobs_2020_2021.jld")

In [ ]:
yTN = file_dict["dec_mar_2021"]
xTN_ = file_dict["ras_mar_2021"]
xTN = xTN_.*cos.(deg2rad.(yTN/3600))

@show xTN_(x_OR6)/3600, yTN(x_OR6)/3600
@show xTN_(x_OR7)/3600, yTN(x_OR7)/3600

# Jacobian of transformation from initial conditions to xTN, yTN
t_x02xyTN = hcat(TaylorSeries.gradient(xTN), TaylorSeries.gradient(yTN))
@show size(t_x02xyTN)

# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_xyTN_OR7 = Hermitian(t_x02xyTN(x_OR7)'*Γ_OR7*(t_x02xyTN(x_OR7)))
Γ_xyTN_OR6 = Hermitian(t_x02xyTN(x_OR6)[1:6,:]'*Γ_OR6*(t_x02xyTN(x_OR6)[1:6,:]))
# get cholesky-L factor
L_xyTN_OR7 = cholesky(Γ_xyTN_OR7).L
L_xyTN_OR6 = cholesky(Γ_xyTN_OR6).L

@show sqrt.(diag(Γ_xyTN_OR6))
@show sqrt.(diag(Γ_xyTN_OR7))

xyTN_v_OR6 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    xyTN_v_OR6[i,:] = L_xyTN_OR6*randn(2)
    xyTN_v_OR7[i,:] = L_xyTN_OR7*randn(2)
end

rndv_2sph_xTN = 3*cos.(angv)
rndv_2sph_yTN = 3*sin.(angv);

xTN_3σ_ell_OR6 = (L_xyTN_OR6[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6 = (L_xyTN_OR6[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6[2,2]*rndv_2sph_yTN)
#
xTN_3σ_ell_OR7 = (L_xyTN_OR7[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7 = (L_xyTN_OR7[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7[2,2]*rndv_2sph_yTN);

ΔxTN_OR76 = xTN(x_OR7)-xTN(x_OR6)
ΔyTN_OR76 = yTN(x_OR7)-yTN(x_OR6)

#plotly()
scatter( ΔxTN_OR76.+xyTN_v_OR7[:,1], ΔyTN_OR76.+xyTN_v_OR7[:,2], ms=0.1, msc=:auto, label="OR7 (JT sampling)")
scatter!(xyTN_v_OR6[:,1], xyTN_v_OR6[:,2], ms=0.1, msc=:auto, label="OR6 (JT sampling)", legend=:outertopright)
#scatter([0], [0], label="OR6 nominal")
scatter!([ΔxTN_OR76], [ΔyTN_OR76], label="OR7 nominal")
plot!(xTN_3σ_ell_OR6, yTN_3σ_ell_OR6, label="OR6 (3-σ)")
plot!(ΔxTN_OR76.+xTN_3σ_ell_OR7, ΔyTN_OR76.+yTN_3σ_ell_OR7, label="OR7 (3-σ)")
xlabel!("RA*cos(DEC) [arcsec]")
ylabel!("DEC [arcsec]")
#yca = year(julian2datetime(jd0 + t_2020(x_OR7)))
#mca = monthname(julian2datetime(jd0 + t_2020(x_OR7)))
title!("RA/DEC observations predictions \n (MPC obs. code 568)")

In [ ]:
### data retrieved from https://newton.spacedys.com/neodys/index.php?n=99942&pc=1.1.4.1&oc=568&y0=2020&m0=10&d0=12&h0=0&mi0=00&s=3.0
radec_3σ_neodys = [0.0176296168   0.0113027277
   0.0069728534   0.0271792443
  -0.0039518729   0.0420112778
  -0.0147247309   0.0552288415
  -0.0249317258   0.0663239917
  -0.0341806085   0.0748703481
  -0.0421159495   0.0805394791
  -0.0484327982   0.0831135230
  -0.0528884017   0.0824935609
  -0.0553115335   0.0787034174
  -0.0556090741   0.0718887458
  -0.0537695892   0.0623114300
  -0.0498637691   0.0503395209
  -0.0440417123   0.0364330922
  -0.0365271572   0.0211265602
  -0.0276088840   0.0050081464
  -0.0176296168  -0.0113027277
  -0.0069728534  -0.0271792443
   0.0039518729  -0.0420112778
   0.0147247309  -0.0552288415
   0.0249317258  -0.0663239917
   0.0341806085  -0.0748703481
   0.0421159495  -0.0805394791
   0.0484327982  -0.0831135230
   0.0528884017  -0.0824935609
   0.0553115335  -0.0787034174
   0.0556090741  -0.0718887458
   0.0537695892  -0.0623114300
   0.0498637691  -0.0503395209
   0.0440417123  -0.0364330922
   0.0365271572  -0.0211265602
   0.0276088840  -0.0050081464
   0.0176296168   0.0113027277];

In [ ]:
angs = range(0.0, stop=2.01pi, length=100)
cos_angs = cos.(angs)
sin_angs = sin.(angs)

a_ = 5.141446E-02/2 # long axis size (arcsec)
b_ = 5.473262E-03/2 # short axis size (arcsec)
cos_ = -0.54278 # long axis u_x
sin_ = 0.83987 # long axis u_y

x_neodys = (a_*cos_).*cos_angs .- (b_*sin_).*sin_angs
y_neodys = (a_*sin_).*cos_angs .+ (b_*cos_).*sin_angs

radec_3σ_neodys = 3 .* [x_neodys y_neodys];

In [ ]:
#plotly()
scatter(xyTN_v_OR6[:,1], xyTN_v_OR6[:,2], ms=0.1, msc=:auto, label="OR6 (JT sampling)")
#scatter([0], [0], label="OR6 nominal")
plot!(xTN_3σ_ell_OR6, yTN_3σ_ell_OR6, label="OR6 (3-σ)")
plot!(radec_3σ_neodys[:,1], radec_3σ_neodys[:,2], label="NEODyS (3-σ)")

xlabel!("RA*cos(DEC) [arcsec]")
ylabel!("DEC [arcsec]")
#yca = year(julian2datetime(jd0 + t_2020(x_OR7)))
#mca = monthname(julian2datetime(jd0 + t_2020(x_OR7)))
title!("RA/DEC observations predictions (MPC obs. code 568)")
#xlims!(-0.1,0.1)
#ylims!(-0.1,0.1)
#xticks!(-0.1:0.02:0.1)
#yticks!(-0.14:0.02:0.14)

# Predicted radar observations: 2021 at Goldstone

In [ ]:
Apophis.loadjpleph()

In [ ]:
Apophis.delay_doppler2(253, DateTime(2021,3,6), 2380.0, xva=Apophis.apophis_pv_199, lod=false, eocorr=false)

In [ ]:
xTN = file_dict["del_mar_2021"]
yTN = file_dict["dop_mar_2021"]

@show xTN(x_OR6), yTN(x_OR6)
@show xTN(x_OR7), yTN(x_OR7)

# Jacobian of transformation from initial conditions to xTN, yTN
t_x02xyTN = hcat(TaylorSeries.gradient(xTN), TaylorSeries.gradient(yTN))
@show size(t_x02xyTN)

# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_xyTN_OR7 = Hermitian(t_x02xyTN(x_OR7)'*Γ_OR7*(t_x02xyTN(x_OR7)))
Γ_xyTN_OR6 = Hermitian(t_x02xyTN(x_OR6)[1:6,:]'*Γ_OR6*(t_x02xyTN(x_OR6)[1:6,:]))
# get cholesky-L factor
L_xyTN_OR7 = cholesky(Γ_xyTN_OR7).L
L_xyTN_OR6 = cholesky(Γ_xyTN_OR6).L

@show sqrt.(diag(Γ_xyTN_OR6))
@show sqrt.(diag(Γ_xyTN_OR7))

xyTN_v_OR6 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    xyTN_v_OR6[i,:] = L_xyTN_OR6*randn(2)
    xyTN_v_OR7[i,:] = L_xyTN_OR7*randn(2)
end

rndv_2sph_xTN = 3*cos.(angv)
rndv_2sph_yTN = 3*sin.(angv);

xTN_3σ_ell_OR6 = (L_xyTN_OR6[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6 = (L_xyTN_OR6[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6[2,2]*rndv_2sph_yTN)
#
xTN_3σ_ell_OR7 = (L_xyTN_OR7[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7 = (L_xyTN_OR7[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7[2,2]*rndv_2sph_yTN);

ΔxTN_OR76 = xTN(x_OR7)-xTN(x_OR6)
ΔyTN_OR76 = yTN(x_OR7)-yTN(x_OR6)

#plotly()
scatter( ΔxTN_OR76.+xyTN_v_OR7[:,1], ΔyTN_OR76.+xyTN_v_OR7[:,2], ms=0.1, msc=:auto, label="OR7 (JT sampling)")
scatter!(xyTN_v_OR6[:,1], xyTN_v_OR6[:,2], ms=0.1, msc=:auto, label="OR6 (JT sampling)")
#scatter([0], [0], label="OR6 nominal")
scatter!([ΔxTN_OR76], [ΔyTN_OR76], label="OR7 nominal")
plot!(xTN_3σ_ell_OR6, yTN_3σ_ell_OR6, label="OR6 (3-σ)")
plot!(ΔxTN_OR76.+xTN_3σ_ell_OR7, ΔyTN_OR76.+yTN_3σ_ell_OR7, label="OR7 (3-σ)")
xlabel!("Delay [us]")
ylabel!("Doppler [Hz]")
#yca = year(julian2datetime(jd0 + t_2020(x_OR7)))
#mca = monthname(julian2datetime(jd0 + t_2020(x_OR7)))
title!("Goldstone March 2021 radar observations predictions")

# Cartesian coordinates to Keplerian elements transformation

In [ ]:
ss16asteph_0 = ss16asteph((jd0-J2000)*daysec)
xs0 = ss16asteph_0[union(3su-2:3su,3(N-1+su)-2:3(N-1+su))]

In [ ]:
Apophis.initialcond(dq)

In [ ]:
ϵ0_deg = 84381.448/3600

In [ ]:
m_eq2ecl = Rx(deg2rad(ϵ0_deg))

In [ ]:
m_xv_eq2ecl = hcat(vcat(m_eq2ecl, zeros(3,3)), vcat(zeros(3,3), m_eq2ecl))

In [ ]:
q0_197 = zeros(6)
q0_197[1:3] .= [-0.9633018953468989, 0.5100289806011301, 0.16528024397505386]
q0_197[4:6] .= [-0.0071188720570829036, -0.012061235689040535, -0.0046695143453363164]
q0_197

In [ ]:
#xas0 = m_xv_eq2ecl*( q0_197 + (jt_scaling.*dq)[1:6] - xs0 ) # taking S197 initial conditions, useful to check vs JPL elements
xas0 = m_xv_eq2ecl*(  Apophis.initialcond(jt_scaling.*dq)[1:6] - xs0  )

In [ ]:
# Solution #197 elements

EPOCH_JPL197 = 2454733.5
EC_JPL197 = .1911952942528226;  QR_JPL197 = .7460724385331012;  TP_JPL197 = 2454894.912507658200
OM_JPL197 = 204.4460284242489;  W_JPL197 = 126.401880836064;  IN_JPL197 = 3.331369228495799
kep_JPL197 = [EC_JPL197, QR_JPL197, TP_JPL197, OM_JPL197, W_JPL197, IN_JPL197]

In [ ]:
ec0 = eccentricity(xas0..., μ[su], 0.0)
a0 = semimajoraxis(xas0..., μ[su], 0.0)
qr0 = a0*(1-ec0)
tp0 = timeperipass(2.4547335e6, xas0..., μ[su], 0.0)
Ω0 = rad2deg(longascnode(xas0...))
ω0 = rad2deg(argperi(xas0..., μ[su], 0.0))
i0 = rad2deg(inclination(xas0...))
A20 = jt_scaling[7]*dq[7]
kep_OR6 = [ec0(x_OR6), qr0(x_OR6), tp0(x_OR6), Ω0(x_OR6), ω0(x_OR6), i0(x_OR6), A20(x_OR6)]
kep_OR7 = [ec0(x_OR7), qr0(x_OR7), tp0(x_OR7), Ω0(x_OR7), ω0(x_OR7), i0(x_OR7), A20(x_OR7)]
kep_197 = [ec0(), qr0(), tp0(), Ω0(), ω0(), i0(), A20()]

In [ ]:
(kep_JPL197 .- kep_OR6[1:6])

In [ ]:
(kep_JPL197 .- kep_197[1:6])./eps.(kep_JPL197)

In [ ]:
kep0 = [ec0, qr0, tp0, Ω0, ω0, i0, A20];

In [ ]:
TaylorSeries.jacobian(kep0)'

In [ ]:
t_car2kep(x) = TaylorSeries.jacobian(kep0, x)'
t_car2kep() = TaylorSeries.jacobian(kep0)'

In [ ]:
t_car2kep()

In [ ]:
t_car2kep() .- TaylorSeries.jacobian(kep0)'

In [ ]:
Γ_O7_kep = (t_car2kep(x_O7)')*Γ_O7*(t_car2kep(x_O7));
Γ_O6_kep = (t_car2kep(x_O6)[1:6,1:6]')*Γ_O6*t_car2kep(x_O6)[1:6,1:6];
Γ_OR7_kep = (t_car2kep(x_OR7)')*Γ_OR7*(t_car2kep(x_OR7));
Γ_OR6_kep = (t_car2kep(x_OR6)[1:6,1:6]')*Γ_OR6*t_car2kep(x_OR6)[1:6,1:6];

In [ ]:
for (x, Γ_kep) in [(x_O6, Γ_O6_kep), (x_OR6, Γ_OR6_kep), (x_O7, Γ_O7_kep), (x_OR7, Γ_OR7_kep)]
    @show kep0(x)
    @show sqrt.(diag(Γ_kep))
    println("* ----===OOO===--- *")
end

In [ ]:
kep0(x_OR7)

In [ ]:
sqrt.(diag(Γ_OR7_kep))

In [ ]:
kep0(x_OR6)

In [ ]:
sqrt.(diag(Γ_OR6_kep))

In [ ]:
Γ_kep_JPL199 = [ 2.858062040774483E-17 -2.531903029559859E-17   2.451271598753632E-14     5.341143529000106E-14     -5.064059304794175E-14     -2.316088496620918E-16     -9.159706240530915E-23;
-2.531903029559859E-17 2.250437378457998E-17   -2.122266051203493E-14     -5.100944859184798E-14     4.826834522209508E-14     2.76636664162405E-16     7.744081722922498E-23;
 2.451271598753632E-14 -2.122266051203493E-14   2.501437350446983E-11     1.16213242311946E-11     -1.057653022558682E-11     4.597741246907787E-13     -1.026132427113056E-19;
 5.341143529000106E-14 -5.100944859184798E-14   1.16213242311946E-11     4.437388022822618E-10     -4.338840441053813E-10     -6.817498150073878E-12     -4.324000765783613E-21;
-5.064059304794175E-14 4.826834522209508E-14   -1.057653022558682E-11     -4.338840441053813E-10     4.261149273549589E-10     6.697032883689781E-12     1.139615135464918E-20;
-2.316088496620918E-16 2.76636664162405E-16   4.597741246907787E-13     -6.817498150073878E-12     6.697032883689781E-12     1.22672053287042E-13     -2.524941685627351E-21;
-9.159706240530915E-23 7.744081722922498E-23   -1.026132427113056E-19     -4.324000765783613E-21     1.139615135464918E-20     -2.524941685627351E-21     4.846398125111792E-28]


In [ ]:
Γ_OR7_kep

In [ ]:
(Γ_kep_JPL199 .- Γ_OR7_kep)./Γ_kep_JPL199

In [ ]:
Γ_OR7_kep ./ Γ_kep_JPL199

In [ ]:
function adotyark(A2, a, e, mu)
    return 2A2/(sqrt(a)*(1-e^2)*sqrt(mu))
end

In [ ]:
kep_OR7

In [ ]:
orbel_OR7_TN = kep_OR7 .+ dq

In [ ]:
ady_kep_au_My = 1e6yr*adotyark(orbel_OR7_TN[7], orbel_OR7_TN[2]/(1.0 - orbel_OR7_TN[1]), orbel_OR7_TN[1], μ[1]) # au/Myr

In [ ]:
ady_kep_m_y = 1e3au*yr*adotyark(orbel_OR7_TN[7], orbel_OR7_TN[2]/(1.0 - orbel_OR7_TN[1]), orbel_OR7_TN[1], μ[1]) #m/yr

In [ ]:
t_kep2ady(x, ady_kep) = TaylorSeries.gradient(ady_kep)(x)

In [ ]:
sqrt( (t_kep2ady(kep_OR7, ady_kep_au_My)')*Γ_OR7_kep*t_kep2ady(kep_OR7, ady_kep_au_My) )

In [ ]:
sqrt( (t_kep2ady(kep_OR7, ady_kep_m_y)')*Γ_OR7_kep*t_kep2ady(kep_OR7, ady_kep_m_y) )

$(-22 \pm 10)\times 10^{-4}$au/Myr


$(-339 \pm 158)$m/yr

## adot OR6 vs OR7 (2029) <---

# Gnuplot

In [ ]:
using Gnuplot

## 2029 B-plane, zoom out

In [ ]:
nomps = 0.08

@gp "unset grid" :-
#@gp :- "set si sq" :-
@gp :- "set size ratio -1" :-
@gp :- "set key top right outside" :- # inside box" :-
#@gp :- xrange=[1.48, 1.50] :-
#@gp :- yrange=[7.1, 7.6] :-
@gp :- "set xlabel '\$\\xi\$ [\$R_\\oplus\$]'" :-
@gp :- "set ylabel '\$\\zeta\$ [\$R_\\oplus\$]'" :-

@gp :- "set style fill transparent solid 0.35 noborder" :-
@gp :- "set style circle radius $nomps" :-

#@gp :-B_dot_ξv_O6*au/RE B_dot_ζv_O6*au/RE "pt 7 ps 0.2 lt 4 t 'JT O6'" :-
#@gp :- B_ξ_3σ_ell_O6 B_ζ_3σ_ell_O6 "w l lw 3 lt 4 t 'O6 3-\$\\sigma\$'" :-
#@gp :- B_dot_ξ(x_O6)*au/RE B_dot_ζ(x_O6)*au/RE "pt 7 ps 0.75 lt 4  t 'O6 nominal'" :-

#@gp :- B_dot_ξv_O7*au/RE B_dot_ζv_O7*au/RE "pt 7 ps 0.2 lt 1 t 'JT O7'" :-
#@gp :- B_ξ_3σ_ell_O7 B_ζ_3σ_ell_O7 "w l lw 3 lt 1 t 'O7 3-\$\\sigma\$'" :-
#@gp :- B_dot_ξ(x_O7)*au/RE B_dot_ζ(x_O7)*au/RE "pt 7 ps 0.75 lt 1  t 'O7 nominal'" :-

#@gp :- B_dot_ξv_OR6*au/RE B_dot_ζv_OR6*au/RE "pt 7 ps 0.2 lt 3 t 'JT OR6'" :-
#@gp :- B_ξ_3σ_ell_OR6 B_ζ_3σ_ell_OR6 "w l lw 3 lt 3 t 'OR6 3-\$\\sigma\$'" :-
#@gp :- B_dot_ξ(x_OR6)*au/RE B_dot_ζ(x_OR6)*au/RE "pt 7 ps 0.75 lt 3 t 'OR6 nominal'" :-

#@gp :- B_dot_ξv_OR7*au/RE B_dot_ζv_OR7*au/RE "pt 7 ps 0.2 lt 2 t 'JT OR7'" :-

#@gp :- cos.(angv) sin.(angv) "w l lt 6 dashtype 2 t'Earth radius'" :-
@gp :- B_dot_ξ(x_OR7)*au/RE B_dot_ζ(x_OR7)*au/RE "w circle lc rgb 'white' fs solid 1.0 border lt -1 t 'OR7 nominal'" :-
@gp :- B_ξ_5σ_ell_OR7 B_ζ_5σ_ell_OR7 "w l lw 2 lc rgb 'gray' t 'OR7 5-\$\\sigma\$'" :-
@gp :- B_ξ_3σ_ell_OR7 B_ζ_3σ_ell_OR7 "w l lw 2 lt 2 lc rgb 'black' t 'OR7 3-\$\\sigma\$'" :-

@gp :- b_E_2029()*cos.(angv) b_E_2029()*sin.(angv) "w l lw 2 lc rgb 'black' dashtype 1 t'Earth cross section'" :-
#@gp :- R_Valsecchi_6_5*cos.(angv) (D_Valsecchi_6_5.+R_Valsecchi_6_5*sin.(angv)) "w l lt 3 t'Valsecchi circle 6:5'" :-
@gp :- R_Valsecchi_8_7*cos.(angv) (D_Valsecchi_8_7.+R_Valsecchi_8_7*sin.(angv)) "w l lw 2 lc rgb 'black' dashtype 3 t 'Valsecchi circle 8:7'" :-
@gp :- R_Valsecchi_7_6*cos.(angv) (D_Valsecchi_7_6.+R_Valsecchi_7_6*sin.(angv)) "w l lw 2 lc rgb 'black' dashtype 6 t 'Valsecchi circle 7:6'"

#Gnuplot.save(term="epslatex standalone color dashed size 7in,6in font \",12\"", output="bplane_2029_RE__.tex")
#run(`pdflatex bplane_2029_RE__.tex`)

## 2029 B-plane, zoom IN

In [ ]:
jtps = 0.25
ellw = 3.0
nomps = 1.5
angv_vc = (pi/4):0.0025:(pi/2);

@gp "unset grid" :-
#@gp "set si sq" :-
#@gp :- "set size ratio 1.2" :-
@gp :- "set key top right outside" :-
@gp :- xrange=[9440, 9520] :-
@gp :- yrange=[45000, 51000] :-
@gp :- "set xlabel '\$\\xi\$ [km]'" :-
@gp :- "set ylabel '\$\\zeta\$ [km]'" :-

@gp :- "set xtics 20" :-

@gp :- "set style fill transparent solid 0.35 noborder" :-
@gp :- "set style circle radius $nomps" :-

@gp :- B_dot_ξv_O7*au B_dot_ζv_O7*au "pt 7 ps $jtps lt 4 t 'JT O7'" :-
@gp :- B_dot_ξv_O6*au B_dot_ζv_O6*au "pt 7 ps $jtps lt 1 t 'JT O6'" :-
@gp :- B_dot_ξv_OR6*au B_dot_ζv_OR6*au "pt 7 ps $jtps lt 3 t 'JT OR6'" :-
@gp :- B_dot_ξv_OR7*au B_dot_ζv_OR7*au "pt 7 ps $jtps lc rgb 'black' t 'JT OR7'" :-

@gp :- B_ξ_5σ_ell_OR7*RE B_ζ_5σ_ell_OR7*RE "w l lw $ellw lc rgb 'gray' t 'OR7 5-\$\\sigma\$'" :-
@gp :- B_ξ_3σ_ell_OR7*RE B_ζ_3σ_ell_OR7*RE "w l lw $ellw lt 2 lc rgb 'black' t 'OR7 3-\$\\sigma\$'" :-
@gp :- B_ξ_3σ_ell_OR6*RE B_ζ_3σ_ell_OR6*RE "w l lw $ellw lt 3 t 'OR6 3-\$\\sigma\$'" :-
@gp :- B_ξ_3σ_ell_O6*RE B_ζ_3σ_ell_O6*RE "w l lw $ellw lt 1 t 'O6 3-\$\\sigma\$'" :-
@gp :- B_ξ_3σ_ell_O7*RE B_ζ_3σ_ell_O7*RE "w l lw $ellw lt 4 t 'O7 3-\$\\sigma\$'" :-

@gp :- B_dot_ξ(x_O6)*au B_dot_ζ(x_O6)*au "w circle lt 1 fs solid 1.0 border lt -1 t 'O6 nominal'" :-
@gp :- B_dot_ξ(x_O7)*au B_dot_ζ(x_O7)*au "w circle lt 4 fs solid 1.0 border lt -1  t 'O7 nominal'" :-
@gp :- B_dot_ξ(x_OR6)*au B_dot_ζ(x_OR6)*au "w circle lt 3 fs solid 1.0 border lt -1 t 'OR6 nominal'" :-
@gp :- B_dot_ξ(x_OR7)*au B_dot_ζ(x_OR7)*au "w circle lc rgb 'white' fs solid 1.0 border lt -1 t 'OR7 nominal'" :-

@gp :- RE*R_Valsecchi_8_7*cos.(angv_vc) RE*(D_Valsecchi_8_7.+R_Valsecchi_8_7*sin.(angv_vc)) "w l lw 2 lc rgb 'black' dashtype 3 t 'Valsecchi circle 8:7'" :-
@gp :- RE*R_Valsecchi_7_6*cos.(angv_vc) RE*(D_Valsecchi_7_6.+R_Valsecchi_7_6*sin.(angv_vc)) "w l lw 2 lc rgb 'black' dashtype 6 t 'Valsecchi circle 7:6'"
#@gp :- RE*R_Valsecchi_6_5*cos.(angv_vc) RE*(D_Valsecchi_6_5.+R_Valsecchi_6_5*sin.(angv_vc)) "w l lw 2 lc rgb 'black' dashtype 4 t 'Valsecchi circle 6:5'"

#@gp :- cos.(angv) sin.(angv) "w l lt 6 dashtype 2 t'Earth radius'" :-
#@gp :- b_E_2029()*cos.(angv) b_E_2029()*sin.(angv) "w l lw 2 lc rgb 'black' dashtype 6 t'Earth cross section'"

#Gnuplot.save(term="epslatex standalone color dashed size 7in,6in font \",12\"", output="bplane_2029_RE_zoom__.tex")
#run(`pdflatex bplane_2029_RE_zoom__.tex`)

## Mar 2021 RA/DEC obs prediction, relative to nominal OR6 orbit

In [ ]:
Gnuplot.options.gpviewer = true

jtps = 0.25
ellw = 3.0
nomps = 1.5
angv_vc = (pi/4):0.0025:(pi/2);

yTN = file_dict["dec_mar_2021"]
xTN_ = file_dict["ras_mar_2021"]
xTN = xTN_.*cos.(deg2rad.(yTN/3600))

@show xTN_(x_OR6)/3600, yTN(x_OR6)/3600
@show xTN_(x_OR7)/3600, yTN(x_OR7)/3600

# Jacobian of transformation from initial conditions to xTN, yTN
t_x02xyTN = hcat(TaylorSeries.gradient(xTN), TaylorSeries.gradient(yTN))
@show size(t_x02xyTN)

# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_xyTN_OR7 = Hermitian(t_x02xyTN(x_OR7)'*Γ_OR7*(t_x02xyTN(x_OR7)))
Γ_xyTN_OR6 = Hermitian(t_x02xyTN(x_OR6)[1:6,:]'*Γ_OR6*(t_x02xyTN(x_OR6)[1:6,:]))
# get cholesky-L factor
L_xyTN_OR7 = cholesky(Γ_xyTN_OR7).L
L_xyTN_OR6 = cholesky(Γ_xyTN_OR6).L

@show sqrt.(diag(Γ_xyTN_OR6))
@show sqrt.(diag(Γ_xyTN_OR7))

xyTN_v_OR6 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    xyTN_v_OR6[i,:] = L_xyTN_OR6*randn(2)
    xyTN_v_OR7[i,:] = L_xyTN_OR7*randn(2)
end

rndv_2sph_xTN = 3*cos.(angv)
rndv_2sph_yTN = 3*sin.(angv);

xTN_3σ_ell_OR6 = (L_xyTN_OR6[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6 = (L_xyTN_OR6[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6[2,2]*rndv_2sph_yTN)
#
xTN_3σ_ell_OR7 = (L_xyTN_OR7[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7 = (L_xyTN_OR7[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7[2,2]*rndv_2sph_yTN);

ΔxTN_OR76 = xTN(x_OR7)-xTN(x_OR6)
ΔyTN_OR76 = yTN(x_OR7)-yTN(x_OR6)

@gp "set multiplot"
@gp :- "unset grid" :-
@gp :- "set si sq" :-
#@gp :- "set size ratio -1" :-
@gp :- "set key top right outside" :-
#@gp :- xrange=[-5,2] :-
#@gp :- yrange=[-1,2] :-
#@gp :- "set xlabel '\$\\alpha\\cos(\\delta)-\\alpha_\\text{OR6}\\cos(\\delta_\\text{OR6})\$ [arcsec]'" :-
#@gp :- "set ylabel '\$\\delta-\\delta_\\text{OR6}\$ [arcsec]'" :-
@gp :- "set xlabel '\$\\alpha\\cos(\\delta)\$ [arcsec]'" :-
@gp :- "set ylabel '\$\\delta\$ [arcsec]'" :-


#@gp :- "set xtics 20" :-

#@gp :- "set style fill transparent solid 0.35 noborder" :-
#@gp :- "set style circle radius $nomps" :-

@gp :- 1 ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 t '7DOF (MC sampling)'" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 t '6DOF (MC sampling)'" :-

@gp :- ΔxTN_OR76.+xTN_3σ_ell_OR7 ΔyTN_OR76.+yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' t '7DOF (3-\$\\sigma\$)'" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 t '6DOF (3-\$\\sigma\$)'" :-

#@gp :- radec_3σ_neodys[:,1] radec_3σ_neodys[:,2] "w l lw $ellw lt 4 t 'NEODyS (3-\$\\sigma\$)'" :-

@gp :- ΔxTN_OR76 ΔyTN_OR76 "w circle lc rgb 'white' fs solid 1.0 border lt -1 t '7DOF (nominal)'" :-

@gp :- 2 "set origin 0.32, 0.50" :-
@gp :- "set size 0.4, 0.45" :-
@gp :- xrange=[-0.1, 0.1] :-
@gp :- yrange=[-0.2, 0.2] :-
@gp :- "unset xlabel" :-
@gp :- "unset ylabel" :-
@gp :- "set xtics 0.1" :-
@gp :- "set ytics 0.2" :-

@gp :- ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 not" :-
@gp :- ΔxTN_OR76.+xTN_3σ_ell_OR7 ΔyTN_OR76.+yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not" #:-
#@gp :- radec_3σ_neodys[:,1] radec_3σ_neodys[:,2] "w l lw $ellw lt 4 not"
#@gp :- "unset multiplot"

Gnuplot.save(term="epslatex standalone color dashed size 7in,5in font \",12\"", output="radec_2020_DDays.tex")
run(`pdflatex radec_2020_DDays.tex`)

Gnuplot.options.gpviewer = false;

## March 2021 Goldstone radar (delay/Doppler) prediction, relative to nominal OR6 orbit

In [ ]:
Gnuplot.options.gpviewer = true

jtps = 0.25
ellw = 3.0
nomps = 1.5
angv_vc = (pi/4):0.0025:(pi/2);

xTN = file_dict["del_mar_2021"]
yTN = file_dict["dop_mar_2021"]

@show xTN(x_OR6), yTN(x_OR6)
@show xTN(x_OR7), yTN(x_OR7)

# Jacobian of transformation from initial conditions to xTN, yTN
t_x02xyTN = hcat(TaylorSeries.gradient(xTN), TaylorSeries.gradient(yTN))
@show size(t_x02xyTN)

# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_xyTN_OR7 = Hermitian(t_x02xyTN(x_OR7)'*Γ_OR7*(t_x02xyTN(x_OR7)))
Γ_xyTN_OR6 = Hermitian(t_x02xyTN(x_OR6)[1:6,:]'*Γ_OR6*(t_x02xyTN(x_OR6)[1:6,:]))
# get cholesky-L factor
L_xyTN_OR7 = cholesky(Γ_xyTN_OR7).L
L_xyTN_OR6 = cholesky(Γ_xyTN_OR6).L

@show sqrt.(diag(Γ_xyTN_OR6))
@show sqrt.(diag(Γ_xyTN_OR7))

xyTN_v_OR6 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    xyTN_v_OR6[i,:] = L_xyTN_OR6*randn(2)
    xyTN_v_OR7[i,:] = L_xyTN_OR7*randn(2)
end

rndv_2sph_xTN = 3*cos.(angv)
rndv_2sph_yTN = 3*sin.(angv);

xTN_3σ_ell_OR6 = (L_xyTN_OR6[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6 = (L_xyTN_OR6[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6[2,2]*rndv_2sph_yTN)
#
xTN_3σ_ell_OR7 = (L_xyTN_OR7[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7 = (L_xyTN_OR7[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7[2,2]*rndv_2sph_yTN);

xTN_2p5σ_ell_OR7 = (2.5/3)*xTN_3σ_ell_OR7
yTN_2p5σ_ell_OR7 = (2.5/3)*yTN_3σ_ell_OR7;

ΔxTN_OR76 = xTN(x_OR7)-xTN(x_OR6)
ΔyTN_OR76 = yTN(x_OR7)-yTN(x_OR6)

@gp "set multiplot"
@gp :- "unset grid" :-
@gp :- "set si sq" :-
#@gp :- "set size ratio -1" :-
@gp :- "set key top right outside" :-
@gp :- xrange=[-1600,600] :-
@gp :- yrange=[-2,5] :-
@gp :- "set xlabel 'Time-delay [\\textmu s]'" :-
@gp :- "set ylabel 'Doppler shift [Hz]'" :-


#@gp :- "set xtics 20" :-

#@gp :- "set style fill transparent solid 0.35 noborder" :-
#@gp :- "set style circle radius $nomps" :-

@gp :- 1 ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 t '7DOF (MC sampling)'" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 t '6DOF (MC sampling)'" :-

@gp :- ΔxTN_OR76.+xTN_3σ_ell_OR7 ΔyTN_OR76.+yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' t '7DOF (3-\$\\sigma\$)'" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 t '6DOF (3-\$\\sigma\$)'" :-

@gp :- ΔxTN_OR76.+xTN_2p5σ_ell_OR7 ΔyTN_OR76.+yTN_2p5σ_ell_OR7 "w l lw $ellw lc rgb 'red' t '7DOF (2.5-\$\\sigma\$)'" :-

@gp :- ΔxTN_OR76 ΔyTN_OR76 "w circle lc rgb 'white' fs solid 1.0 border lt -1 t '7DOF (nominal)'" :-
@gp :- [1.1243345490830308e8].-xTN(x_OR6) [6121.1974930763245].-yTN(x_OR6) "pt 1 ps 2.5 lw 4 lt 4 t 'JPL 199 (nominal)'" :-

@gp :- 2 "set origin 0.28, 0.475" :-
@gp :- "set size 0.45, 0.5" :-
@gp :- xrange=[-11, 11] :-
@gp :- yrange=[-0.12, 0.12] :-
@gp :- "unset xlabel" :-
@gp :- "unset ylabel" :-
@gp :- "set xtics 5" :-
@gp :- "set ytics 0.05" :-

@gp :- ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 not" :-
@gp :- ΔxTN_OR76.+xTN_3σ_ell_OR7 ΔyTN_OR76.+yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not"
#@gp :- "unset multiplot"

Gnuplot.save(term="epslatex standalone color dashed size 7in,5in font \",12\"", output="deldop_2021_Goldstone_DDays.tex")
run(`pdflatex deldop_2021_Goldstone_DDays.tex`)

Gnuplot.options.gpviewer = false;

## April 2029 closest approach (CA) time and distance, relative to nominal OR6 orbit

In [ ]:
Gnuplot.options.gpviewer = true

jtps = 0.25
ellw = 3.0
nomps = 1.5
angv_vc = (pi/4):0.0025:(pi/2);

xTN = rae_2029*au
yTN = t_2029*daysec

@show xTN(x_OR6), yTN(x_OR6)
@show xTN(x_OR7), yTN(x_OR7)

# Jacobian of transformation from initial conditions to xTN, yTN
t_x02xyTN = hcat(TaylorSeries.gradient(xTN), TaylorSeries.gradient(yTN))
@show size(t_x02xyTN)

# transform cov matrix to d_CA_2020, t_CA_2020 coords
Γ_xyTN_OR7 = Hermitian(t_x02xyTN(x_OR7)'*Γ_OR7*(t_x02xyTN(x_OR7)))
Γ_xyTN_OR6 = Hermitian(t_x02xyTN(x_OR6)[1:6,:]'*Γ_OR6*(t_x02xyTN(x_OR6)[1:6,:]))
# get cholesky-L factor
L_xyTN_OR7 = cholesky(Γ_xyTN_OR7).L
L_xyTN_OR6 = cholesky(Γ_xyTN_OR6).L

@show sqrt.(diag(Γ_xyTN_OR6))
@show sqrt.(diag(Γ_xyTN_OR7))

xyTN_v_OR6 = Matrix{Float64}(undef, 1000, 2)
xyTN_v_OR7 = Matrix{Float64}(undef, 1000, 2)
for i in 1:1000
    xyTN_v_OR6[i,:] = L_xyTN_OR6*randn(2)
    xyTN_v_OR7[i,:] = L_xyTN_OR7*randn(2)
end

rndv_2sph_xTN = 3*cos.(angv)
rndv_2sph_yTN = 3*sin.(angv);

xTN_3σ_ell_OR6 = (L_xyTN_OR6[1,1]*rndv_2sph_xTN .+ L_xyTN_OR6[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR6 = (L_xyTN_OR6[2,1]*rndv_2sph_xTN .+ L_xyTN_OR6[2,2]*rndv_2sph_yTN)
#
xTN_3σ_ell_OR7 = (L_xyTN_OR7[1,1]*rndv_2sph_xTN .+ L_xyTN_OR7[1,2]*rndv_2sph_yTN)
yTN_3σ_ell_OR7 = (L_xyTN_OR7[2,1]*rndv_2sph_xTN .+ L_xyTN_OR7[2,2]*rndv_2sph_yTN);

ΔxTN_OR76 = xTN(x_OR7)-xTN(x_OR6)
ΔyTN_OR76 = yTN(x_OR7)-yTN(x_OR6)

@gp "set multiplot"
@gp :- "unset grid" :-
@gp :- "set si sq" :-
#@gp :- "set size ratio -1" :-
@gp :- "set key top right outside" :-
@gp :- xrange=[-1800,1200] :-
@gp :- yrange=[-35,10] :-
@gp :- "set xlabel 'Geocentric range at closest approach [km]'" :-
@gp :- "set ylabel 'Time of closest approach [seconds]'" :-

#@gp :- "set xtics 20" :-

#@gp :- "set style fill transparent solid 0.35 noborder" :-
#@gp :- "set style circle radius $nomps" :-

@gp :- 1 ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 t 'OR7 (JT sampling)'" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 t 'OR6 (JT sampling)'" :-

@gp :- ΔxTN_OR76.+xTN_3σ_ell_OR7 ΔyTN_OR76.+yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' t 'OR7 3-\$\\sigma\$'" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 t 'OR6 3-\$\\sigma\$'" :-

@gp :- ΔxTN_OR76 ΔyTN_OR76 "w circle lc rgb 'white' fs solid 1.0 border lt -1 t 'OR7 nominal'" :-

@gp :- 2 "set origin 0.325, 0.125" :-
@gp :- "set size 0.4, 0.45" :-
@gp :- xrange=[-23, 23] :-
@gp :- yrange=[-1, 1] :-
@gp :- "unset xlabel" :-
@gp :- "unset ylabel" :-
@gp :- "set xtics 20" :-
@gp :- "set ytics 0.5" :-

@gp :- ΔxTN_OR76.+xyTN_v_OR7[:,1] ΔyTN_OR76.+xyTN_v_OR7[:,2] "pt 7 ps $jtps lt -1 not" :-
@gp :- ΔxTN_OR76.+xTN_3σ_ell_OR7 ΔyTN_OR76.+yTN_3σ_ell_OR7 "w l lw $ellw lc rgb 'black' not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not" :-
@gp :- xyTN_v_OR6[:,1] xyTN_v_OR6[:,2] "pt 7 ps $jtps lt 3 not" :-
@gp :- xTN_3σ_ell_OR6 yTN_3σ_ell_OR6 "w l lw $ellw lt 3 not"
#@gp :- "unset multiplot"

Gnuplot.save(term="epslatex standalone color dashed size 7in,5in font \",12\"", output="dt_CA_2029.tex")
run(`pdflatex dt_CA_2029.tex`)

Gnuplot.options.gpviewer = false;

In [ ]:
# multiplot example below to be run in REPL

# 2020/2021/2029 RA/dec predictions

In [ ]:
Apophis.loadjpleph()

In [ ]:
et0 = (jd0-J2000)*daysec

In [ ]:
sun_dofs = union(3su-2:3su,3(27+su)-2:3(27+su))
earth_dofs = union(3ea-2:3ea,3(27+ea)-2:3(27+ea));

In [ ]:
sun_et(et) = Apophis.auday2kmsec( ss16asteph(et)[sun_dofs] )
earth_et(et) = Apophis.auday2kmsec( ss16asteph(et)[earth_dofs] )

In [ ]:
tv1

In [ ]:
yr0 = 2000 + (jd0 - J2000)/yr

In [ ]:
jdv = jd0 .+ tv1
dtv = julian2datetime.(jdv)[2:end-460];

In [ ]:
dtv[end]

In [ ]:
alphav = Vector{Float64}(undef, length(dtv))
deltav = Vector{Float64}(undef, length(dtv))
tauv = Vector{Float64}(undef, length(dtv))
nuv = Vector{Float64}(undef, length(dtv))

for i in eachindex(dtv)
    alphav[i] , deltav[i] = Apophis.radec(251, dtv[i], pm=false, lod=false, eocorr=false, xvs=sun_et, xve=earth_et)
    tauv[i], nuv[i] = Apophis.delay_doppler2(251, dtv[i], 2380, pm=false, lod=false, eocorr=false, xvs=sun_et, xve=earth_et)
end

In [ ]:
#plot(tv1[2:end-460], alphav)
yrv = yr0 .+ tv1[2:end-460]/yr
plot(yrv, deltav/3600, label=false)
plot!([yrv[1], yrv[end]], [-1,-1]) #, label="Lower decl. lim.")
plot!([yrv[1], yrv[end]], [38,38]) #, label="Upper decl. lim.")
ylims!(-50,50)
ylabel!("Declination [deg]")
xlabel!("Year")

In [ ]:
#plot(tv1[2:end-460], alphav)
yrv = yr0 .+ tv1[2:end-460]/yr
#plot(yrv, 1e-6clightkms*tauv/(2au))
plot(yrv, tauv/1e9)
plot!([yrv[1], yrv[end]], [2.3522085507e8,2.3522085507e8]/1e9)
#ylims!(0,2.5)
ylims!(0,2.5e9/1e9)
#ylabel!("cτ/2 [au]")
ylabel!("Time-delay [10⁹us]")
xlabel!("Year")

In [ ]:
(.824-.751)*365.25

In [ ]:
.07*365.25

In [ ]:
.2*365.25-31-28

In [ ]:
.119*365.25-31

- LocalRegistries.jl

# Float64 vs Float128

In [ ]:
dx = xv1(x_OR7) .- xv1_f64(x_OR7);

In [ ]:
plot(tv1, au*dx[:,1])
plot!(tv1, au*dx[:,2])
plot!(tv1, au*dx[:,3])

In [ ]:
function atan(y::Taylor1, x::Taylor1)
    a = atan(y/x)
    a[0] = atan(constant_term(y), constant_term(x))
    return a
end